# dp100_16 AzureMLを使用してバッチ推論パイプラインをデプロイする

大量のデータを操作する実行期間の長いタスクが"バッチ"操作として実行される。  
機械学習では"バッチ推論"を使用して、予測モデルが複数のケースに非同期に適用される。

AzureMLでバッチ推論ソリューションを実装するには、入力データを読み取り、登録済みのモデルを読み込み、  
ラベルを予測し、結果をその出力として書き込むためのステップを含むパイプラインを作成する。

## バッチ推論パイプラインを作成する

### 1.モデルを登録する

バッチ推論パイプラインでトレーニング済みモデルを使用するには、それを自分のAzureMLワークスペースに登録する必要がある。  
ローカルファイルからモデルを登録するには、**Model**オブジェクトの**register**メソッドを使用する。

```
from azureml.core import Model

classification_model = Model.register(workspace=your_workspace,
                                      model_name='classification_model',
                                      model_path='model.pkl', # local path
                                      description='A classification model')
```

また、モデルのトレーニングに使用される**Run**への参照がある場合は、**register_model**メソッドを使用できる。

```
run.register_model( model_name='classification_model',
                    model_path='outputs/model.pkl', # run outputs path
                    description='A classification model')
```

### 2.スコアリングスクリプトを作成する

バッチ推論サービスには、モデルを読み込んでからそれを使用して新しい値を予測するスコアリングスクリプトが必要。  
次の2つの関数が含まれている必要がある。

- **init()** : パイプラインが初期化されると呼び出される
- **run(mini_batch)** : 処理するデータのバッチ毎に呼び出される

通常は、**init**関数を使用してモデルレジストリからモデルを読み込む。  
また、**run**関数を使用してデータの各パッチから予測を生成して結果を返す。

```
import os
import numpy as np
from azureml.core import Model
import joblib

def init():
    # Runs when the pipeline step is initialized
    global model

    # load the model
    model_path = Model.get_model_path('classification_model')
    model = joblib.load(model_path)

def run(mini_batch):
    # 各バッチ毎に実行される
    resultList = []

    # バッチ内の各ファイルを処理する
    for f in mini_batch:
        # カンマで区切られたデータを配列に読み込む
        data = np.genfromtxt(f, delimiter=',')
        # モデル入力用の2次元配列に整形
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        # 予測値を結果に加える
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList
```

### 3.ParallelRunStepを使用してパイプラインを作成する

AzureMLには、並列バッチ推論を実行するための一種のパイプラインステップが特に用意されている。  
**ParallelRunStep**クラスを使用すると、**File**データセットからファイルのバッチを読み取り、  
処理出力を**PipelineData**参照に書き込むことができる。  
さらに、そのステップの**output_action**設定を"append_row"に設定することもできる。  
これにより、並列に実行されているステップのすべてのインスタンスで、  
それぞれの結果が確実に*parallel_run_step.txt*という名前の1つの出力ファイルと照合される。  
次のコードスニペットは、**ParallelRunStep**を使用してパイプラインを作成する例を示している。

```
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.pipeline.core import PipelineData
from azureml.pipeline.core import Pipeline

# Get the batch dataset for input
batch_data_set = ws.datasets['batch-data']

# Set the output location
default_ds = ws.get_default_datastore()
output_dir = PipelineData(name='inferences',
                          datastore=default_ds,
                          output_path_on_compute='results')

# Define the parallel run step step configuration
parallel_run_config = ParallelRunConfig(
    source_directory='batch_scripts',
    entry_script="batch_scoring_script.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=aml_cluster,
    node_count=4)

# Create the parallel run step
parallelrun_step = ParallelRunStep(
    name='batch-score',
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('batch_data')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)
# Create the pipeline
pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
```

### 4.パイプラインを実行してステップの出力を取得する

パイプラインが定義されたら、それを実行して完了するまで待つ。  
さらに以下のコード例のように、ステップの出力から**parallel_run_step.txt**ファイルを取得して結果を表示できる。

```
from azureml.core import Experiment

# パイプラインを実験として実行
pipeline_run = Experiment(ws, 'batch_prediction_pipeline').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

# 最初の(そして唯一の)ステップからの出力を取得
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='results')

# parallel_run_step.txtファイルの検索
for root, dirs, files in os.walk('results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# 結果の出力
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]
print(df)
```

## バッチ推論パイプラインを発行する

バッチ推論パイプラインはRESTサービスとして発行することができる。

```
published_pipeline = pipeline_run.publish_pipeline(name='Batch_Prediction_Pipeline',
                                                   description='Batch pipeline',
                                                   version='1.0')
rest_endpoint = published_pipeline.endpoint
```

発行されたら、サービスエンドポイントを使用してバッチ推論ジョブを開始できる。

```
import requests

response = requests.post(rest_endpoint,
                         headers=auth_header,
                         json={"ExperimentName": "Batch_Prediction"})
run_id = response.json()["Id"]
```

また、発行されたパイプラインが自動的に実行されるようにスケジュールすることもできる。

```
from azureml.pipeline.core import ScheduleRecurrence, Schedule

weekly = ScheduleRecurrence(frequency='Week', interval=1)
pipeline_schedule = Schedule.create(ws, name='Weekly Predictions',
                                        description='batch inferencing',
                                        pipeline_id=published_pipeline.id,
                                        experiment_name='Batch_Prediction',
                                        recurrence=weekly)
```

## 演習 バッチ推論パイプラインを作成する

### ワークスペースの接続

In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.28.0 to work with 20210613


### モデルの訓練と登録

In [2]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name='mslearn-train-diabetes')
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

Starting experiment: mslearn-train-diabetes
Loading Data...
Training a decision tree model
Accuracy: 0.8903333333333333
AUC: 0.8780859744265883
Model trained and registered.


### バッチデータの生成とアップロード

演習用に、すでにあるデータからランダムサンプリングし、そのデータをAzureMLワークスペースのデータストアにアップロードしてデータセットを登録する。

In [3]:
from azureml.core import Datastore, Dataset
import pandas as pd
import os

# Set default data store
ws.set_default_datastore('workspaceblobstore')
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

# Load the diabetes data
diabetes = pd.read_csv('data/diabetes2.csv')
# Get a 100-item sample of the feature columns (not the diabetic label)
sample = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].sample(n=100).values

# Create a folder
batch_folder = './batch-data'
os.makedirs(batch_folder, exist_ok=True)
print("Folder created!")

# Save each sample as a separate file
print("Saving files...")
for i in range(100):
    fname = str(i+1) + '.csv'
    sample[i].tofile(os.path.join(batch_folder, fname), sep=",")
print("files saved!")

# Upload the files to the default datastore
print("Uploading files to datastore...")
default_ds = ws.get_default_datastore()
default_ds.upload(src_dir="batch-data", target_path="batch-data", overwrite=True, show_progress=True)

# Register a dataset for the input data
batch_data_set = Dataset.File.from_files(path=(default_ds, 'batch-data/'), validate=False)
try:
    batch_data_set = batch_data_set.register(workspace=ws, 
                                             name='batch-data',
                                             description='batch data',
                                             create_new_version=True)
except Exception as ex:
    print(ex)

print("Done!")

azureml_globaldatasets - Default = False
workspaceblobstore - Default = True
workspacefilestore - Default = False
Folder created!
Saving files...
files saved!
Uploading files to datastore...
Uploading an estimated of 100 files
Uploading batch-data/1.csv
Uploaded batch-data/1.csv, 1 files out of an estimated total of 100
Uploading batch-data/10.csv
Uploaded batch-data/10.csv, 2 files out of an estimated total of 100
Uploading batch-data/100.csv
Uploaded batch-data/100.csv, 3 files out of an estimated total of 100
Uploading batch-data/11.csv
Uploaded batch-data/11.csv, 4 files out of an estimated total of 100
Uploading batch-data/12.csv
Uploaded batch-data/12.csv, 5 files out of an estimated total of 100
Uploading batch-data/13.csv
Uploaded batch-data/13.csv, 6 files out of an estimated total of 100
Uploading batch-data/14.csv
Uploaded batch-data/14.csv, 7 files out of an estimated total of 100
Uploading batch-data/15.csv
Uploaded batch-data/15.csv, 8 files out of an estimated total of 1

### コンピューティングの作成

パイプラインにはコンピューティングコンテキストが必要なので、コンピューティングクラスタを指定する。

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "msl-20210613b"

try:
    # Check for existing compute target
    inference_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        inference_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        inference_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)    

Found existing cluster, use it.


### バッチ推論パイプラインの作成

パイプラインにはバッチ推論を実行するためのPythonコードが必要なので、パイプラインで使用するすべてのファイルを保管するフォルダを作成する。

In [5]:
import os
# Create a folder for the experiment files
experiment_folder = 'batch_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

batch_pipeline


実際の作業を行うPythonスクリプトを作成し、pipelineフォルダに保存する。

In [6]:
%%writefile $experiment_folder/batch_diabetes.py
import os
import numpy as np
from azureml.core import Model
import joblib


def init():
    # Runs when the pipeline step is initialized
    global model

    # load the model
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)


def run(mini_batch):
    # This runs for each batch
    resultList = []

    # process each file in the batch
    for f in mini_batch:
        # Read the comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        # Reshape into a 2-dimensional array for prediction (model expects multiple items)
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

Writing batch_pipeline/batch_diabetes.py


次に、スクリプトが必要となる依存関係を含む実行コンテキストを定義する。

In [7]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.runconfig import CondaDependencies

# モデルに必要な依存関係の追加
# scikit-learnモデルにはscikit-learnが必要
# 並列パイプラインのステップには、azureml-coreとazureml-dataprep[fuse]が必要
cd = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                              pip_packages=['azureml-defaults','azureml-core','azureml-dataprep[fuse]'])

batch_env = Environment(name='batch_environment')
batch_env.python.conda_dependencies = cd
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print('Configuration ready.')

Configuration ready.


バッチ予測スクリプトを実行し、入力データから予測値を生成し、その結果をテキストファイルとして出力フォルダに保存するパイプラインを使用することになる。  
このためにはParallelRunStepを使用する。これにより、バッチデータを並行して処理し、  
結果を*parallel_run_step.txt*という単一の出力ファイルにまとめることができる。

> 注:"enabled"は非推奨であるという渓谷が表示されることがあるが、無視して問題ない

In [8]:
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.pipeline.core import PipelineData
from azureml.core.runconfig import DockerConfiguration

default_ds = ws.get_default_datastore()

output_dir = PipelineData(name='inferences', 
                          datastore=default_ds, 
                          output_path_on_compute='diabetes/results')

parallel_run_config = ParallelRunConfig(
    source_directory=experiment_folder,
    entry_script="batch_diabetes.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=inference_cluster,
    node_count=2)

parallelrun_step = ParallelRunStep(
    name='batch-score-diabetes',
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('diabetes_batch')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

print('Steps defined')

Steps defined


あとは、このステップをパイプラインに入れて実行するのみ　※時間がかかる

In [9]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
pipeline_run = Experiment(ws, 'mslearn-diabetes-batch').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

Created step batch-score-diabetes [156ead4b][e6f294ef-33b2-4307-8139-34be78d6b6fc], (This step will run and generate new outputs)
Submitted PipelineRun f2ae0c56-645d-4323-a5e6-eea5d180d89f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/f2ae0c56-645d-4323-a5e6-eea5d180d89f?wsid=/subscriptions/153404fd-72ab-4092-b50e-de490c5509fc/resourcegroups/20210613/workspaces/20210613&tid=5456e8d8-0223-4619-ba5b-e313627da53d
PipelineRunId: f2ae0c56-645d-4323-a5e6-eea5d180d89f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/f2ae0c56-645d-4323-a5e6-eea5d180d89f?wsid=/subscriptions/153404fd-72ab-4092-b50e-de490c5509fc/resourcegroups/20210613/workspaces/20210613&tid=5456e8d8-0223-4619-ba5b-e313627da53d
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: cd739555-37f2-4496-a40d-2cb55edb7c5c
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/cd739555-37f2-4496-a40d-2cb55edb7c5c?wsid=/subscriptions/153404fd-72ab-4092-b50e-de490c5509fc/res

'Finished'

パイプラインの実行が終了すると、結果として得られた予測値はパイプラインの最初の(そして唯一の)ステップに関連する実験の出力に保存される。  
それは以下のようにして取り出すことができる。

In [10]:
import pandas as pd
import shutil

# 前回の実行結果が残っている場合は、ローカルの結果フォルダを削除
shutil.rmtree('diabetes-results', ignore_errors=True)

# 最初のステップのRunを取得し、その出力をダウンロード
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='diabetes-results')

# フォルダ階層をたどって結果ファイルを探す
for root, dirs, files in os.walk('diabetes-results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# 出y録フォーマットのクリーンナップ
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# 結果の出力(先頭20件)
df.head(20)

,File,Prediction
0,13.csv,1
1,14.csv,0
2,15.csv,1
3,16.csv,0
4,17.csv,0
5,27.csv,0
6,28.csv,0
7,29.csv,1
8,3.csv,1
9,30.csv,0


### パイプラインの公開とRESTインターフェイスの使用

バッチ推論パイプラインが完成したので、これを公開してRESTエンドポイントを用いてアプリケーションから実行することができる。

In [11]:
published_pipeline = pipeline_run.publish_pipeline(
    name='diabetes-batch-pipeline', description='Batch scoring of diabetes data', version='1.0')

published_pipeline

Name,Id,Status,Endpoint
diabetes-batch-pipeline,891d6831-3cc8-4273-9820-cacc27b2dbf5,Active,REST Endpoint


> 注 : 公開されたパイプラインにはエンドポイントがあり、Azureポータルで確認できる。  
エンドポイントは、公開されたパイプラインオブジェクトのプロパティからも確認できる。

In [12]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://westus2.api.azureml.ms/pipelines/v1.0/subscriptions/153404fd-72ab-4092-b50e-de490c5509fc/resourceGroups/20210613/providers/Microsoft.MachineLearningServices/workspaces/20210613/PipelineRuns/PipelineSubmit/891d6831-3cc8-4273-9820-cacc27b2dbf5


エンドポイントを利用するには、クライアントからHTTPでRESTコールをする必要がある。  
このリクエストは認証が必要なのでauthorizationヘッダが必要で、これをテストするためにAzureワークスペースへの現在の接続からの認証ヘッダーを使用する。

> 注 : 実際のアプリケーションでは、認証されるためのサービスプリンシパルが必要になる。

In [13]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print('Authentication header ready.')

Authentication header ready.


これで、RESTインターフェイスを呼び出す準備ができた。  
バッチ推論パイプラインは非同期的に実行されるので、識別子が返ってくる。  
この識別子を用いて、パイプラインの実験が実行されているかどうかを追跡することができる。

In [14]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "mslearn-diabetes-batch"})
run_id = response.json()["Id"]
run_id

'5d5d8353-1e0b-4d24-a838-4ab544c0b170'

RunIDがあるので、**RunDetails**ウィジェットを使って実験が実行されている様子を見ることができる。

In [15]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments['mslearn-diabetes-batch'], run_id)

# Block until the run completes
published_pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: 5d5d8353-1e0b-4d24-a838-4ab544c0b170
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/5d5d8353-1e0b-4d24-a838-4ab544c0b170?wsid=/subscriptions/153404fd-72ab-4092-b50e-de490c5509fc/resourcegroups/20210613/workspaces/20210613&tid=5456e8d8-0223-4619-ba5b-e313627da53d
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: bc6d3475-e8af-4aa7-864f-d20f4f2343c7
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/bc6d3475-e8af-4aa7-864f-d20f4f2343c7?wsid=/subscriptions/153404fd-72ab-4092-b50e-de490c5509fc/resourcegroups/20210613/workspaces/20210613&tid=5456e8d8-0223-4619-ba5b-e313627da53d

StepRun(batch-score-diabetes) Execution Summary
StepRun( batch-score-diabetes ) Status: Finished
{'runId': 'bc6d3475-e8af-4aa7-864f-d20f4f2343c7', 'target': 'msl-20210613b', 'status': 'Completed', 'startTimeUtc': '2021-06-15T09:42:38.585768Z', 'endTimeUtc': '2021-06-15T09:42:38.713083Z', 'properties': {'azureml.reusedrunid': 'cd739555-37f2-4496-a40

'Finished'

パイプラインの実行が完了するのを待ってから、以下のセルを実行して結果を確認する。  
先程と同様に、結果は最初のパイプラインステップの出力にある。

In [17]:
import pandas as pd
import shutil

# 前回の実行結果が残っている場合、ローカルの結果フォルダを削除
shutil.rmtree('diabetes-results', ignore_errors=True)

# 最初のステップのRunを取得し、その出力をダウンロード
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='diabetes-results')

# フォルダ階層をたどって結果ファイルを探す
for root, dirs, files in os.walk('diabetes-results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# 出力形式をクリーンナップ
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# 結果の出力
df.head(20)

,File,Prediction
0,13.csv,1
1,14.csv,0
2,15.csv,1
3,16.csv,0
4,17.csv,0
5,27.csv,0
6,28.csv,0
7,29.csv,1
8,3.csv,1
9,30.csv,0


これで、バッチ処理するためのパイプラインが出来上がった。

バッチ推論の詳細 : https://docs.microsoft.com/azure/machine-learning/how-to-run-batch-predictions

## 知識チェック

1. あなたは大量のデータ ファイルの新しい値を予測するために使用するバッチ推論パイプラインを作成中ですか?  
あなたはパイプラインによってスコアリング スクリプトが複数のノード上で実行され、その結果が照合されるようにしたいと考えています。  
パイプラインにはどのような種類のステップを含める必要がありますか?
    - PythonScriptStep
    - ParallelRunStep
    - AdlaStep


2. あなたは output_action="append_row" プロパティを使用してバッチ推論パイプライン内のステップを構成しました。  
どのファイルでバッチ推論の結果を探す必要がありますか?
    - output.txt
    - stdoutlogs.txt
    - parallel_run_step.txt

↓解答

1. ParallelRunStep
    - ParallelRunStep ステップを使用して、スコアリング スクリプトを並列に実行する必要があります。
2. parallel_run_step.txt
    - append_row 出力アクションを使用すると、ParallelRunStep ステップからの結果が、  
    parallel_run_step.txt という名前のファイル内で照合されます。